In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
df = pd.read_csv("Resources/lending_data.csv")

# Review the DataFrame
df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [3]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = df['loan_status']

# Separate the X variable, the features
X = df.drop(columns = ["loan_status"])

In [4]:
# Review the y variable Series
y.head()

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [5]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [6]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [7]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [8]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
model = LogisticRegression(solver = 'lbfgs', random_state = 1, max_iter = 1000)

# Fit the model using training data
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [9]:
# Make a prediction using the testing data
prediction = model.predict(X_test)
prediction

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [10]:
# Print the balanced_accuracy score of the model
print(balanced_accuracy_score(y_test, prediction))

0.9672620331306306


In [11]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, prediction)

array([[18658,   107],
       [   37,   582]], dtype=int64)

In [12]:
# Print the classification report for the model
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.84      0.94      0.89       619

    accuracy                           0.99     19384
   macro avg       0.92      0.97      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:**  Of 19,384 records tested, our first model correctly identified 18,658 of the 18,695 "0", mislabeling 37 and correctly labeling pertneer 100% of the healthy loans, correctly identifying 582 of the 689 "1", mislabeling 107 and correctly labeling 84% of the risky loans (our `Precision Score`[^1]). This result makes sense because the original dataset only has a fraction as many "1" records as "0" records: y.value_counts() = 75036 "0" and 2500 "1". Despite mislabeling a higher percentage of "1" than "0", 582 of the 619 records labeled "1" did belong to that category, giving us a `Recall Score` of 94%[^1].  When class distributions are imbalanced, our `F1 Score` of 89%[^1] best describes the overall accuracy of this model.[^2] **This model can be trusted to predict which loans will be healthy vs risky with 89% accuracy.** 

[^1] https://medium.com/@kennymiyasato/classification-report-precision-recall-f1-score-accuracy-16a245a437a5 <br>
[^2] https://medium.com/analytics-vidhya/accuracy-vs-f1-score-6258237beca2#:~:text=Accuracy%20can%20be%20used%20when,to%20evaluate%20our%20model%20on

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [14]:
# First install imbalanced-learn
!pip install -U imbalanced-learn 

# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
model = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_resampled, y_resampled = model.fit_resample(X_train, y_train)

   ---------------------------------------- 257.7/257.7 kB 1.8 MB/s eta 0:00:00


In [15]:
# Count the distinct values of the resampled labels data
y_resampled.value_counts()

0    56271
1    56271
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [16]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
model = LogisticRegression(random_state=1)

# Fit the model using the resampled training data
model.fit(X_resampled, y_resampled)

# Make a prediction using the testing data
prediction = model.predict(X_resampled)
prediction

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [17]:
# Print the balanced_accuracy score of the model 
print(balanced_accuracy_score(y_resampled, prediction))

0.9946686570347071


In [18]:
# Generate a confusion matrix for the model
print(confusion_matrix(y_resampled, prediction))

[[55957   314]
 [  286 55985]]


In [19]:
# Print the classification report for the model
print(classification_report(y_resampled, prediction))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     56271
           1       0.99      0.99      0.99     56271

    accuracy                           0.99    112542
   macro avg       0.99      0.99      0.99    112542
weighted avg       0.99      0.99      0.99    112542



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:**  Of 112542 records tested, our second model "correctly" identified 55,957 of the 56,271 "0", mislabeling 286 and "correctly" labeling pertneer 99% of the healthy loans and "correctly" identified 55,985 of the 56,299 "1", mislabeling 314 and "correctly" labeling 99% of the risky loans (our `Precision Score`[^1]). 55,985 of the 56,271 records labeled "1" did belong to that category, giving us a 99% `Recall Score`[^1].  Now that class distributions are artificially balanced, our 99% `Accuracy Score`[^1] best describes the overall "accuracy" of this model.[^2] The original dataset of 75,536 records was imbalanced with a mere 2,500 records labeled "1" (risky). With only 3% of the records available for the `test-train split` (2,500 / 75,536 = 0.0330968), the fact that the original model could be trusted to predict which loans would be risky with an 89% accuracy rate is a testament to the power of the `LogisticRegression` model. To correct the imbalance, our `RandomOverSampler` bloated the file by 45% ((112,542 - 75,536) / 75,536 = 0.451481) because it "duplicates examples from the minority class in the training dataset and can result in overfitting for some models."[^3] The 37,0006 clones represent over 14 times the number of actual risky loans (112,542 - 75,536 = 37,006 / 2,500 = 14.8024), rendering this model a self-congratulating exercise with no practical use. **This model should NOT be trusted to predict any real world activity.**

[^1] https://medium.com/@kennymiyasato/classification-report-precision-recall-f1-score-accuracy-16a245a437a5 <br>
[^2] https://medium.com/analytics-vidhya/accuracy-vs-f1-score-6258237beca2#:~:text=Accuracy%20can%20be%20used%20when,to%20evaluate%20our%20model%20on <br>
[^3] https://machinelearningmastery.com/random-oversampling-and-undersampling-for-imbalanced-classification/